In [1]:
import pandas as pd
from common import Data

In [11]:
df = Data.get_nwsl_matches()
df = df[~df["is_playoffs"]]
df = df.sort_values("date")
df

,date,time,season,home,away,home_score,away_score,home_penalty_score,away_penalty_score,home_xg,away_xg,referee,attendance,stadium,is_forfeit,is_extra_time,is_pks,is_playoffs
0,2013-04-13,19:35,2013,KC,POR,1,1,0.0,0.0,NaN,NaN,Kari Seitz,6784.0,Shawnee Mission District Stadium,False,False,False,False
1,2013-04-14,17:00,2013,CHI,RGN,1,1,0.0,0.0,NaN,NaN,Josh Wilkens,1255.0,Benedictine U. Sports Complex Stadium,False,False,False,False
2,2013-04-14,18:00,2013,NJNY,WNY,1,0,0.0,0.0,NaN,NaN,John McCloskey,2611.0,Yurcak Field,False,False,False,False
3,2013-04-14,18:30,2013,BOS,WAS,1,1,0.0,0.0,NaN,NaN,Hernan Aguilar,2634.0,Dilboy Stadium,False,False,False,False
4,2013-04-20,19:00,2013,WAS,WNY,1,1,0.0,0.0,NaN,NaN,Kari Seitz,4569.0,Maryland SoccerPlex,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,2024-04-26,19:00,2024,LA,KCC,1,3,0.0,0.0,1.0,2.9,Katja Koroleva,19178.0,BMO Stadium,False,False,False,False
1320,2024-04-27,18:30,2024,CHI,POR,0,2,0.0,0.0,1.4,1.3,Thomas Snyder,4544.0,SeatGeek Stadium,False,False,False,False
1321,2024-04-27,19:00,2024,NC,RGN,1,0,0.0,0.0,1.5,1.5,Gerald Flores,8135.0,Sahlen's Stadium at WakeMed Soccer Park,False,False,False,False
1319,2024-04-27,17:30,2024,UTA,HOU,0,0,0.0,0.0,0.9,0.5,Shawn Tehini,10349.0,America First Field,False,False,False,False


In [24]:
n = 5
result = pd.DataFrame([], columns=["season", "team", "rank", "early_points"])

for season in map(str, range(2013,2024)):
    if season == "2020":
        continue

    df_season = df[df["season"] == season]
    table = Data.get_nwsl_standings(df_season)

    for team in table["team"]:
        position = table[table["team"] == team].index[0]
        count = len(table.index)

        df_team = df_season[(df_season["home"] == team) | (df_season["away"] == team)].iloc[0:n]

        early_points = 0
        for i, row in df_team.iterrows():
            if row["home_score"] == row["away_score"]:
                early_points += 1
            elif row["home_score"] > row["away_score"] and row["home"] == team:
                early_points += 3
            elif row["away_score"] > row["home_score"] and row["away"] == team:
                early_points += 3
                
        result.loc[len(result.index)] = [season, team, (count - position + 1) / count, early_points] 

result

,season,team,rank,early_points
0,2013,WNY,1.000000,7
1,2013,KC,0.875000,10
2,2013,POR,0.750000,13
3,2013,NJNY,0.625000,10
4,2013,BOS,0.500000,8
...,...,...,...,...
93,2023,WAS,0.416667,9
94,2023,LOU,0.333333,4
95,2023,HOU,0.250000,6
96,2023,KCC,0.166667,6


In [26]:
result.to_csv("5_game_points_to_rank.csv", index=False)